# 02 — Water vs land, false positives, and speckle
This notebook helps you inspect the **water-like mask** and manually reason about **false positives**.

It loads:
- `water_mask_t1.tif`
- optional polygon layer (GeoPackage) if you want to inspect vector output

## Basemap overlay
A true basemap overlay in-notebook usually needs extra packages.
This notebook includes an **optional** overlay using `contextily` + `geopandas`.
If you don’t want extra deps, you can still do strong inspection by:
- plotting rasters here, and
- opening the GeoPackages in QGIS with an XYZ basemap.


In [ ]:
from pathlib import Path
import numpy as np
import rasterio
import matplotlib.pyplot as plt

OUTDIR = Path('../outputs')  # change if you ran with --outdir
water_mask_path = OUTDIR / 'water_mask_t1.tif'
ratio_path = OUTDIR / 'ratio_db_t1_minus_t0.tif'

print('water mask:', water_mask_path, 'exists=', water_mask_path.exists())
print('ratio_db:', ratio_path, 'exists=', ratio_path.exists())


## Load and display water mask

In [ ]:
with rasterio.open(water_mask_path) as ds:
    mask = ds.read(1).astype('float32')
    transform = ds.transform
    crs = ds.crs
    extent = rasterio.plot.plotting_extent(ds)

plt.figure(figsize=(7,6))
plt.imshow(mask, vmin=0, vmax=1)
plt.title('Water-like mask (t1)')
plt.colorbar(label='0/1')
plt.axis('off')
plt.show()

print('CRS:', crs)


## Why false positives happen (quick SAR intuition)
Even if the threshold is reasonable, you can get false positives due to:
- **Speckle**: coherent interference makes pixel-level values noisy.
- **Wind roughening on water**: rough water can look brighter (less "water-like").
- **Look angle & incidence** variations.
- **Urban shadows / layover**: geometry effects.
- **Wet soil / smooth surfaces** can resemble low backscatter.

A common way to reduce false positives is to use:
- multi-temporal aggregation (median over several dates)
- speckle filtering (Lee, Frost, etc.)
- VV+VH features


## Optional: overlay on a basemap (requires extra deps)
If you want a basemap overlay inside the notebook, install:
```bash
python -m pip install geopandas contextily shapely pyproj
```
Then run the next cell. If you skip it, use QGIS instead.


In [ ]:
try:
    import geopandas as gpd
    import contextily as cx
    from shapely.geometry import box
    from rasterio.warp import transform_bounds
except Exception as e:
    print('Optional deps not installed:', e)
    print('Install with: python -m pip install geopandas contextily shapely pyproj')
    raise

# Compute bounds in EPSG:3857 for basemap tiles
with rasterio.open(water_mask_path) as ds:
    b = ds.bounds
    src_crs = ds.crs

minx, miny, maxx, maxy = transform_bounds(src_crs, 'EPSG:3857', b.left, b.bottom, b.right, b.top, densify_pts=21)
bbox_poly = box(minx, miny, maxx, maxy)
gdf = gpd.GeoDataFrame({'name':['aoi']}, geometry=[bbox_poly], crs='EPSG:3857')

fig, ax = plt.subplots(figsize=(8,8))
gdf.boundary.plot(ax=ax)
cx.add_basemap(ax, source=cx.providers.OpenStreetMap.Mapnik)
ax.set_title('AOI footprint on basemap (EPSG:3857)')
plt.show()


## Recommended: manual inspection in QGIS
1. Load `outputs/water_polys_t1.gpkg` and/or the raster `outputs/water_mask_t1.tif`
2. Add XYZ basemap (OpenStreetMap)
3. Compare mask against known water bodies and land cover

Look for consistent misclassification patterns and tune `--water-thr-db`.
